# DataJoint Workflow Array Ephys

This notebook will describe the steps for interacting with the data ingested into `workflow-array-ephys`.

In [ ]:
import os
os.chdir('..')

In [ ]:
import datajoint as dj
import matplotlib.pyplot as plt
import numpy as np

from workflow_array_ephys.pipeline import lab, subject, session, ephys

## Workflow architecture

This workflow is assembled from 3 DataJoint elements:
+ [element-lab](https://github.com/datajoint/element-lab)
+ [element-animal](https://github.com/datajoint/element-animal)
+ [element-array-ephys](https://github.com/datajoint/element-array-ephys)

For the architecture and detailed descriptions for each of those elements, please visit the respective links. 

Below is the diagram describing the core components of the fully assembled pipeline.


In [ ]:
dj.Diagram(ephys) + (dj.Diagram(Session) + 1) - 1

## Browsing the data with DataJoint query and fetch 


DataJoint provides abundant functions to query data and fetch. For a detailed tutorials, visit our [general tutorial site](https://playground.datajoint.io/)


Running through the pipeline, we have ingested data of subject6 session1 into the database. Here are some highlights of the important tables.

### `Subject` and `Session` tables

In [ ]:
subject.Subject()

In [ ]:
session.Session()

In [ ]:
session_key = (Session & 'subject="subject6"' & 'session_datetime = "2021-01-15 11:16:38"').fetch1('KEY')

### `ephys.ProbeInsertion` and `ephys.EphysRecording` tables

These tables stores the probe recordings within a particular session from one or more probes.

In [ ]:
ephys.ProbeInsertion & session_key

In [ ]:
ephys.EphysRecording & session_key

### `ephys.ClusteringTask` , `ephys.Clustering`, `ephys.Curation`, and `ephys.CuratedClustering`

+ Spike-sorting is performed on a per-probe basis with the details stored in `ClusteringTask` and `Clustering`

+ After the spike sorting, the results may go through curation process. 
    + If it did not go through curation, a copy of ClusteringTask was inserted into table `ephys.Curation`. 
    + If it did go through a curation, a new entry will be inserted into `ephys.Curation`, with a `curation_output_dir` different from `clustering_output_dir` in table `ephys.ClusteringTask`
    + `ephys.Curation` supports multiple curations of a ClusteringTask.

In [ ]:
ephys.ClusteringTask * ephys.Clustering & session_key

In our example workflow, `curation_output_dir` is the same as `clustering_output_dir`

In [ ]:
ephys.Curation * ephys.CuratedClustering & session_key

### Spike-sorting results are stored in `ephys.CuratedClustering`,  `ephys.WaveformSet.Waveform`

In [ ]:
ephys.CuratedClustering.Unit & session_key

Let's pick one probe and further inspect the clustering results

In [ ]:
insertion_key = (ephys.CuratedClustering & session_key & 'insertion_number = 0').fetch1('KEY')

In [ ]:
ephys.CuratedClustering.Unit & insertion_key

### Generate a raster plot

Let's try a raster plot - just the "good" units

In [ ]:
units, unit_spiketimes = (ephys.CuratedClustering.Unit & insertion_key & 'cluster_quality_label = "good"').fetch('unit', 'spike_times')

In [ ]:
x = np.hstack(unit_spiketimes)
y = np.hstack([np.full_like(s, u) for u, s in zip(units, unit_spiketimes)])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(32, 16))
ax.plot(x, y, '|')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Unit')

### Plot waveform of a unit

Let's pick one unit and further inspect

In [ ]:
unit_key = (ephys.CuratedClustering.Unit & session_key & 'insertion_number = 0' & 'unit = 30').fetch1('KEY')

In [ ]:
ephys.CuratedClustering.Unit * ephys.WaveformSet.Waveform & unit_key

In [ ]:
unit_data = (ephys.CuratedClustering.Unit * ephys.WaveformSet.PeakWaveform & unit_key).fetch1()

In [ ]:
unit_data

In [ ]:
plt.plot(unit_data['peak_electrode_waveform'])

## Summary and Next Step

This notebook highlights the major tables in the workflow and visualize some of the ingested results. 

The next notebook [06-drop](06-drop[optional].ipynb) shows how to drop schemas and tables if needed.